<a href="https://colab.research.google.com/github/giovannimmiranda/DL_20-06_GMM/blob/main/DL_20_06_Miranda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Learning Exam - 20/06/2024**


Giovanni Michele Miranda,  507567

In [128]:
import urllib.request
import zipfile
# ZIP data URL from GitHub
zip_url = "https://github.com/giovannimmiranda/DL_20-06_GMM/raw/main/input_data.zip"
# Download the ZIP
zip_path, _ = urllib.request.urlretrieve(zip_url)
# Unzip the folder
target_folder = "temp"
# reading data
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)
data_path = f"{target_folder}/input_data.pkl"
#Then use the following lines to read it:
import pickle as pk
with open(data_path, 'rb') as f:
  data = pk.load(f)
images = data['imgs']
y = data['labels']
labels = y[:,0]
bboxes = y[:,1:]

In [130]:
bboxes

array([[ 23,  42, 206, 199],
       [ 13,  70, 219, 160],
       [  8,  42, 221, 200],
       [ 22,  67, 218, 158],
       [ 15,  56, 212, 156],
       [ 24,  82, 204, 154],
       [ 40,  89, 216, 221],
       [ 14,  44, 208, 205],
       [ 28,  71, 193, 174],
       [ 53,  86, 149, 175],
       [ 32,  74, 200, 132],
       [  6,  74, 221, 157],
       [ 12,  51, 216, 165],
       [ 16,  82, 208, 172],
       [ 26,  22, 201, 172],
       [  8,  71, 215, 152],
       [ 13, 129, 196, 170],
       [  7,  82, 219, 191],
       [ 31,  73, 209, 142],
       [ 96,  63, 219, 164],
       [ 24,  61, 196, 185],
       [ 14,  97, 214, 184],
       [  8,  38, 219, 180],
       [ 24,  26, 200, 197],
       [ 36,  11, 215, 207],
       [ 74,  37, 150, 187],
       [ 21,  29, 202, 198],
       [ 16,  52, 210, 184],
       [  4,  35, 225, 196],
       [ 29,  74, 195, 121],
       [ 29,  77, 209, 172],
       [ 38,  57, 207, 191],
       [ 37,  25, 192, 180],
       [ 12,  47, 217, 135],
       [ 25,  

In [94]:
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [121]:
if np.max(images) > 1 and np.min(images) > 0:
  images = images.astype('float32') / 255.0
else:
  pass

In [122]:
import numpy as np
import random

# Example labels to remove images from
label_to_remove_c1 = 1  # First class
label_to_remove_c2 = 2  # Second class

# Ensure there are enough images in the dataset
if labels.shape[0] > 184 and images.shape[0] > 184:
    # Find indices of images with the specified labels
    indices_to_remove_c1 = np.where(labels == label_to_remove_c1)[0]
    indices_to_remove_c2 = np.where(labels == label_to_remove_c2)[0]

    # Check if there are enough images to remove

    # Randomly select two indices from the first class
    random_indices_c1 = random.sample(list(indices_to_remove_c1), 2)
    # Randomly select one index from the second class
    random_indices_c2 = random.sample(list(indices_to_remove_c2), 1)

    # Remove the selected images and their corresponding labels
    images = np.delete(images, random_indices_c1, axis=0)
    labels = np.delete(labels, random_indices_c1, axis=0)
    # Adjust the second class index due to the removal of the first class indices
    random_indices_c2 = [i - np.sum(random_indices_c1 < i) for i in random_indices_c2]
    images = np.delete(images, random_indices_c2, axis=0)
    labels = np.delete(labels, random_indices_c2, axis=0)

    print(f"Removed images at indices: {random_indices_c1} from class 1 and {random_indices_c2} from class 2")
    print(f"New dataset size: {images.shape[0]} images, {labels.shape[0]} labels")
else:
    print(f"The dataset is already set! It has {images.shape[0]} images.")
    unique_labels, counts = np.unique(labels, return_counts=True)
    print("\nNow the dataset is balanced:")
    for label, count in zip(unique_labels, counts):
      print(f"\n Class {label}: {count} images")

Removed images at indices: [22, 46] from class 1 and [84] from class 2
New dataset size: 183 images, 183 labels


In [123]:
images.shape

(183, 227, 227, 3)

In [140]:
# Example image dimensions (assuming)
image_width_height = 227
batch_size = 5

# Convert to float for accurate division
bboxes_normalized = bboxes.astype(float)

# Normalize bounding boxes to [0, 1]
bboxes_normalized[:, 0] /= image_width_height  # Normalize x1
bboxes_normalized[:, 1] /= image_width_height  # Normalize y1
bboxes_normalized[:, 2] /= image_width_height  # Normalize x2
bboxes_normalized[:, 3] /= image_width_height  # Normalize y2
print(f"Printing first {batch_size} normalized bounding boxes:")
print(bboxes_normalized[:batch_size])

Printing first 5 normalized bounding boxes:
[[0.10132159 0.18502203 0.90748899 0.87665198]
 [0.05726872 0.30837004 0.96475771 0.70484581]
 [0.03524229 0.18502203 0.97356828 0.88105727]
 [0.0969163  0.29515419 0.96035242 0.69603524]
 [0.0660793  0.24669604 0.9339207  0.68722467]]
